In [ ]:
import requests
import json
from dateutil import parser, tz
from datetime import datetime, timezone


def iso_to_timestamp(isodt_str: str):
    isodt = parser.parse(isodt_str)
    return pd.Timestamp(
        isodt.replace(tzinfo=timezone.utc).astimezone(
            tz=tz.gettz("America/Los_Angeles")
        )
    ).tz_convert(tz=tz.gettz("America/Los_Angeles"))


def request_to_json(request):
    response = requests.get(request)
    response.encoding = "utf-8-sig"
    return response.json()


def convert_time_str_to_local_tz_timestamp(df, time_cols):
    df[time_cols] = df[time_cols].applymap(iso_to_timestamp)
    return df


def departures_response_to_rwc_departures(departures):
    # only if stopping at rwc and going north
    nb_trains = [
        x
        for x in departures["ServiceDelivery"]["StopMonitoringDelivery"][
            "MonitoredStopVisit"
        ]
        if x["MonitoredVehicleJourney"]["DestinationName"]
        == "San Francisco Caltrain Station"
    ]

    rwc_next = pd.DataFrame(
        [
            x["MonitoredVehicleJourney"]["MonitoredCall"]
            | {"VehicleRef": x["MonitoredVehicleJourney"]["VehicleRef"]}
            for x in nb_trains
        ]
    ).loc[lambda df: df.StopPointName == "Redwood City Caltrain Station"]
    rwc_next = convert_time_str_to_local_tz_timestamp(
        rwc_next,
        time_cols=[
            "AimedArrivalTime",
            "ExpectedArrivalTime",
            "AimedDepartureTime",
            "ExpectedDepartureTime",
        ],
    )
    return (
        rwc_next[["VehicleRef", "AimedDepartureTime", "ExpectedDepartureTime"]]
        .assign(
            minutes_late=lambda df: (
                df.ExpectedDepartureTime - df.AimedDepartureTime
            ).apply(lambda x: x.seconds / 60)
        )
        .rename(
            columns={
                "AimedDepartureTime": "scheduled_rwc_departure",
                "ExpectedDepartureTime": "expected_rwc_departure",
                "VehicleRef": "vehicle_id",
            }
        )
    )


# your_key = "66114e41-7929-4a61-a9c5-96cff817e7df"
operatorID = "CT"
rwc_caltrain_id = "redwood_city"

In [ ]:
env_vars = !cat ~/.bash_env_vars

In [ ]:
key = os.environ['CALTRAIN_API_KEY']

# Stop endpoint

In [ ]:
departure_prediction = (
    f"http://api.511.org/transit/StopMonitoring?api_key={key}&agency=CT&stop={rwc_caltrain_id}"
)
departures_response = request_to_json(departure_prediction)

In [ ]:
departures = departures_response_to_rwc_departures(departures_response)

# Vehicle endpoint

In [ ]:
rt_response = request_to_json(
    f"http://api.511.org/transit/VehicleMonitoring?api_key={key}&agency={operatorID}"
)

In [ ]:
# this + 6 minutes
train_data = pd.DataFrame(
    [
        x["MonitoredVehicleJourney"]["OnwardCalls"]["OnwardCall"][-1]
        | {"vehicle_id": x["MonitoredVehicleJourney"]["VehicleRef"]}
        for x in rt_response["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"][
            "VehicleActivity"
        ]
        if x["MonitoredVehicleJourney"]["VehicleRef"] in departures.vehicle_id.values
    ]
)[["StopPointName", "AimedDepartureTime", "ExpectedDepartureTime", "vehicle_id"]]

In [ ]:
train_data

In [ ]:
train_data = convert_time_str_to_local_tz_timestamp(
    train_data, ["AimedDepartureTime", "ExpectedDepartureTime"]
)
sf_arrival = train_data.rename(
    columns={
        "AimedDepartureTime": "scheduled_sf_arrival",
        "ExpectedDepartureTime": "expected_sf_arrival",
        "VehicleRef": "vehicle_id",
    }
).assign(
    scheduled_sf_arrival=lambda df: df.scheduled_sf_arrival + pd.Timedelta("6 minutes"),
    expected_sf_arrival=lambda df: df.expected_sf_arrival + pd.Timedelta("6 minutes"),
).drop(columns=['StopPointName'])

In [ ]:
sf_arrival

In [ ]:
departures.merge(sf_arrival,on='vehicle_id').drop(columns=['vehicle_id'])

# Misc info gathering

In [ ]:
# This gives caltrain operatorID: CT
operators = f"http://api.511.org/transit/operators?api_key={your_key}"
# this gives the name of rwc caltrain stop
stops = f"http://api.511.org/transit/stops?api_key={your_key}&operator_id={operatorID}&format=json"

In [ ]:
response_stops = request_to_json(stops)
[x for x in response_stops.json()['Contents']['dataObjects']['ScheduledStopPoint'] if 'Redwood'.lower() in x['Name'].lower()]